# Server-Side History Storage Example

This example demonstrates how conversation history is managed when using the **Azure AI Agent service**.

**Key Points:**
- Conversation history is stored in the Azure cloud service
- Only a thread reference (ID) is sent with each request
- The service maintains full conversation state
- Network payload remains constant regardless of conversation length

**Required Environment Variables:**
- `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint

**Prerequisites:**
- An existing Azure AI Agent created in Azure AI Foundry
- Appropriate permissions to access Azure AI Agent service

In [ ]:
import os
import asyncio
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder
from dotenv import load_dotenv

load_dotenv()

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
print(f"Project endpoint: {project_endpoint}")

In [12]:
def demonstrate_server_side_history():
    """
    Demonstrates server-side conversation history management.
    The Azure AI Agent service stores conversation history in the cloud.
    """
    print("=== SERVER-SIDE HISTORY STORAGE DEMO ===")
    print("In this approach:")
    print("1. Conversation history is stored in Azure AI Agent service")
    print("2. Only thread ID + new message is sent with each request")
    print("3. Service maintains full conversation state")
    print("4. Network payload remains constant\n")
    
    # Initialize the Azure AI project client
    project = AIProjectClient(
        credential=DefaultAzureCredential(),
        endpoint=project_endpoint
    )
    
    # First, let's list available agents
    print("📋 Listing available agents...")
    try:
        agents_paged = project.agents.list_agents()
        agents_list = list(agents_paged)  # Convert ItemPaged to list
        
        if agents_list:
            print(f"Found {len(agents_list)} agent(s):")
            for i, agent in enumerate(agents_list[:3]):  # Show first 3
                print(f"  {i+1}. {agent.name} (ID: {agent.id})")
            
            # Use the first available agent
            selected_agent = agents_list[0]
            print(f"\n🤖 Using agent: {selected_agent.name} (ID: {selected_agent.id})")
        else:
            print("❌ No agents found. Please create an agent in Azure AI Foundry first.")
            return
    except Exception as e:
        print(f"❌ Error listing agents: {e}")
        print("\n💡 Note: This example requires an existing Azure AI Agent.")
        print("Please create one in Azure AI Foundry portal first.")
        return
    
    # Create a thread in the Azure cloud service
    print(f"\n📝 Creating conversation thread in Azure cloud...")
    thread = project.agents.threads.create()
    print(f"✅ Thread created with ID: {thread.id}")
    print(f"🌐 Thread is stored on Azure servers, not locally\n")
    
    # First interaction
    print("🔄 FIRST REQUEST:")
    user_message_1 = "Can you help me understand machine learning concepts?"
    print(f"User: {user_message_1}")
    
    # Add message to the cloud-stored thread
    message_1 = project.agents.messages.create(
        thread_id=thread.id,  # Only sending thread reference
        role="user",
        content=user_message_1
    )
    print(f"📤 Sent to service: thread_id='{thread.id}' + message (not full history)")
    
    # Run the agent
    run_1 = project.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=selected_agent.id
    )
    
    if run_1.status == "completed":
        messages_paged = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        messages_list = list(messages_paged)  # Convert ItemPaged to list
        latest_response = [msg for msg in messages_list if msg.role == "assistant"][-1]
        if latest_response.text_messages:
            print(f"Assistant: {latest_response.text_messages[0].text.value[:100]}...")
    else:
        print(f"❌ Run failed: {run_1.last_error}")
        return
    
    # Second interaction - builds on cloud-stored context
    print(f"\n🔄 SECOND REQUEST (with context):")
    user_message_2 = "What are the main differences between supervised and unsupervised learning?"
    print(f"User: {user_message_2}")
    
    message_2 = project.agents.messages.create(
        thread_id=thread.id,  # Same thread reference
        role="user",
        content=user_message_2
    )
    print(f"📤 Sent to service: thread_id='{thread.id}' + new message only")
    print(f"🧠 Service retrieves full conversation history from cloud storage")
    
    run_2 = project.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=selected_agent.id
    )
    
    if run_2.status == "completed":
        messages_paged = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        messages_list = list(messages_paged)  # Convert ItemPaged to list
        latest_response = [msg for msg in messages_list if msg.role == "assistant"][-1]
        if latest_response.text_messages:
            print(f"Assistant: {latest_response.text_messages[0].text.value[:100]}...")
    
    # Third interaction - continues with cloud context
    print(f"\n🔄 THIRD REQUEST (with full context):")
    user_message_3 = "Can you give me a practical example of each type?"
    print(f"User: {user_message_3}")
    
    message_3 = project.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message_3
    )
    print(f"📤 Sent to service: thread_id='{thread.id}' + new message only")
    print(f"🧠 Service has access to complete conversation context automatically")
    
    run_3 = project.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=selected_agent.id
    )
    
    if run_3.status == "completed":
        messages_paged = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        messages_list = list(messages_paged)  # Convert ItemPaged to list
        latest_response = [msg for msg in messages_list if msg.role == "assistant"][-1]
        if latest_response.text_messages:
            print(f"Assistant: {latest_response.text_messages[0].text.value[:100]}...")
    
    # Show final conversation summary
    print(f"\n📊 CONVERSATION SUMMARY:")
    all_messages_paged = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    all_messages_list = list(all_messages_paged)  # Convert ItemPaged to list
    print(f"Total messages in thread: {len(all_messages_list)}")
    print(f"Thread ID: {thread.id} (stored in Azure cloud)")
    
    print("\n" + "="*60)
    print("💡 KEY INSIGHTS:")
    print("• Each request sends only thread_id + new message")
    print("• Network payload remains constant (efficient)")
    print("• Service automatically retrieves full context")
    print("• Conversation persists in Azure cloud storage")
    print("• Highly scalable for long conversations")
    print("="*60)

In [13]:
# Run the demonstration
demonstrate_server_side_history()

=== SERVER-SIDE HISTORY STORAGE DEMO ===
In this approach:
1. Conversation history is stored in Azure AI Agent service
2. Only thread ID + new message is sent with each request
3. Service maintains full conversation state
4. Network payload remains constant

📋 Listing available agents...
Found 1 agent(s):
  1. Server Agent (ID: asst_hbGLNHBuZTWhBDiTKlOFnoVU)

🤖 Using agent: Server Agent (ID: asst_hbGLNHBuZTWhBDiTKlOFnoVU)

📝 Creating conversation thread in Azure cloud...
Found 1 agent(s):
  1. Server Agent (ID: asst_hbGLNHBuZTWhBDiTKlOFnoVU)

🤖 Using agent: Server Agent (ID: asst_hbGLNHBuZTWhBDiTKlOFnoVU)

📝 Creating conversation thread in Azure cloud...
✅ Thread created with ID: thread_MQJLVC8bByKofjAx4uQnq4nQ
🌐 Thread is stored on Azure servers, not locally

🔄 FIRST REQUEST:
User: Can you help me understand machine learning concepts?
✅ Thread created with ID: thread_MQJLVC8bByKofjAx4uQnq4nQ
🌐 Thread is stored on Azure servers, not locally

🔄 FIRST REQUEST:
User: Can you help me under

## How Server-Side History Works

### Data Flow Visualization:

```
Initial Setup:
Client → [Create Thread] → Azure AI Service
                             ↓
Client ← [Thread ID: "abc123"] ← Azure AI Service

Request 1:
Client → [thread_id: "abc123", message: "Help with ML"] → Azure AI Service
                                                            ↓ (stores message)
Client ← [Response: "Here's ML overview..."] ← Azure AI Service

Request 2:
Client → [thread_id: "abc123", message: "Supervised vs unsupervised?"] → Azure AI Service
                                                                          ↓ (retrieves full history)
                                                                        [ML overview + new question]
                                                                          ↓
Client ← [Response: "Supervised learning uses labels..."] ← Azure AI Service

Request 3:
Client → [thread_id: "abc123", message: "Give examples?"] → Azure AI Service
                                                            ↓ (has complete context)
                                                          [Full conversation history]
                                                            ↓
Client ← [Response: "Example of supervised: email spam detection..."] ← Azure AI Service
```

### Advantages:
- **Efficient network usage** - constant small payloads
- **Scalable** for very long conversations
- **Persistent storage** - conversations survive client disconnections
- **Automatic state management** - no client-side complexity
- **Shared access** - multiple clients can access same conversation

### Disadvantages:
- **Cloud dependency** - requires connection to Azure
- **Less control** over conversation data
- **Potential costs** for storage and API calls
- **Service availability** dependency

## Creating Your Own Azure AI Agent

To use this example, you'll need to create an Azure AI Agent in Azure AI Foundry:

1. **Go to Azure AI Foundry portal** (https://ai.azure.com)
2. **Navigate to your project**
3. **Go to "Agents" section**
4. **Click "+ Create agent"**
5. **Configure your agent:**
   - Name: "ML Teaching Assistant"
   - Instructions: "You are a helpful assistant that explains machine learning concepts clearly and provides practical examples."
   - Select appropriate model deployment
6. **Save and deploy** your agent

Once created, you can use the agent ID in your code or let the code automatically select the first available agent.